In [1]:
import gdspy
import pandas as pd
import numpy as np
import sys
import math
import pattern_writer as pw

In [2]:
baseUnit = 1 #layout scale(microns)
m=baseUnit*1e6 
nm = m*1E-9

In [3]:
def posArrayGenerator(a, b, numA, numB, generator_function, placement_function):
    arr=[]
    new_x=0
    new_y=0
    for x in range(-numA,numA):
        for y in range(-numB,numB):
            vec=[(x*a[0]+y*b[0], x*a[1]+y*b[1])]
            geometry_cell= generator_function(placement_function(vec[0])
            arr.append(gdspy.CellReference(geometry_cell, (vec[0][0], vec[0][1]), magnification=1))
    return arr

SyntaxError: invalid syntax (<ipython-input-3-5d667817db1b>, line 9)

In [ ]:
geometry1 = gdspy.Cell("CIRCLE")
geometry1.add(gdspy.Round((0, 0), 72*nm, tolerance=0.001))
geometry2 = gdspy.Cell("OVAL")
geometry2.add(gdspy.Round((0, 0), [72*nm, 64*nm], tolerance=1e-4))
def p_f(point):
    return np.sqrt(point[0]**2 + point[1]**2)
def g_f(distance):
    if distance < 500*nm:
        #geometry1 = gdspy.Cell("CIRCLE")
        #geometry1.add(gdspy.Round((0, 0), 93*nm, tolerance=0.001))
        return geometry1
    else:
        #geometry2 = gdspy.Cell("OVAL")
        #geometry2.add(gdspy.Round((0, 0), [93*nm, 80*nm], tolerance=1e-4))
        return geometry2
#y = layout_placement2(230*nm, 570*nm,70,4,6,g_f, p_f)
y = posArrayGenerator([4,2], [5,3], 8, 1 ,g_f, p_f)

In [ ]:
gdspy.current_library = gdspy.GdsLibrary()
Lens = gdspy.Cell("LENS")
Lens.add(y)
gdspy.current_library.add(Lens)
gdspy.write_gds('Testing35.gds')